In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
import json

### Utility 함수들

In [3]:
# utility functions

import json
def get_cost(index1, index2, cost_type, json_path, toll_include=True):
    """
    index1, index2와 cost_type을 지정해주면 json 파일을 파싱하여 비용을 반환
    """

    # 출발지와 도착지가 같은 경우 0 반환
    if index1 == index2:
        return 0
    else:
        from_idx = min(index1, index2)
        to_idx = max(index1, index2)

        # 마지막에 /가 없으면 붙이기
        if json_path[-1] != '/':
            json_path += '/'
        json_path = json_path + f'route_{from_idx}_{to_idx}.json'
        
        try:
            with open(json_path, 'r') as file:
                data = json.load(file)
        except:
            print(f"{json_path} 파일을 읽을 수 없음")
            raise ValueError()
        
        try:
            tollFare = 0 if toll_include else data['route'][cost_type][0]['summary']['tollFare']
            feulPrice = data['route'][cost_type][0]['summary']['fuelPrice']
            return tollFare+feulPrice
        except:
            print(f"{json_path} 파일의 tollFare와 fuelPrice 파싱 불가")
            raise ValueError()

### 수요 및 각 node와 depot의 cost

In [4]:
DEPOT_INDEX = 0
COST_TYPE = 'trafast'
mu = 300
std = 100

node_data = pd.read_csv('./../Data/data.csv')

# 데이터프레임 생성
node_demands = pd.DataFrame(columns=['node'], data=node_data[node_data['index'] != DEPOT_INDEX]['index'].values)

# 각 node에서의 수요(필요에 따라 코드 수정)
node_demands['demand'] = None
for node in node_demands['node']:
    machine_num = node_data.loc[node_data['index']==node, 'machine'].values[0]

    node_demand = 0

    # 기기수만큼 정규분포에서 랜덤 샘플링을 진행하여 더해줌
    while(machine_num != 0):
        node_demand += np.random.normal(mu, std)
        machine_num -= 1

    node_demand = round(node_demand)
    if node_demand < 0:
        node_demand = 0
    node_demands.loc[node_demands['node'] == node, 'demand'] = node_demand

# node와 depot의 cost
node_demands['depot_node_cost'] = node_demands['node'].apply(lambda x : get_cost(DEPOT_INDEX, x, COST_TYPE, './../Data/routes', toll_include=True))

##### Node-link 구조 활용

In [4]:
import sys
sys.path.append('C:/Users/james/OneDrive/바탕 화면/대학교/수업/2024-1/물류관리/Term_project/Modules')
from savings.link import Link
from savings.node import Node
from savings.graph import Graph

In [5]:
DEPOT_INDEX = 0
COST_TYPE = 'trafast'
JSON_PATH = './../Data/routes'
mu = 300
std = 100

node_data = pd.read_csv('./../Data/data.csv')

In [6]:
g = Graph()

for row in node_data.iterrows():
    data = row[1]

    # Node의 demand 생성 로직
    machine_num = data['machine']
    node_demand = 0
    # 기기수만큼 정규분포에서 랜덤 샘플링을 진행하여 더해줌
    while(machine_num != 0):
        node_demand += np.random.normal(mu, std)
        machine_num -= 1
    node_demand = round(node_demand)
    if node_demand < 0:
        node_demand = 0

    # Node 생성 후 등록
    n = Node(data['index'], data['longitude'], data['latitude'], node_demand)
    g.add_node(n)



In [7]:
g.print_nodes()

Node 0 (127.6678744, 34.7610282) : Demand 676
Node 1 (127.6793937, 34.9374162) : Demand 244
Node 2 (127.5852535, 34.976458) : Demand 562
Node 3 (127.5822458, 34.9807918) : Demand 317
Node 4 (127.58105, 34.9734096) : Demand 449
Node 5 (127.7251479, 34.9628515) : Demand 366
Node 6 (127.2885331, 35.2861086) : Demand 196
Node 7 (126.39205, 34.7909583) : Demand 395
Node 8 (127.7220062, 34.9629217) : Demand 131
Node 9 (126.779112, 35.0143833) : Demand 345
Node 10 (127.692538, 34.9513867) : Demand 578
Node 11 (126.3970753, 34.8171651) : Demand 477
Node 12 (127.4867548, 34.9676712) : Demand 404
Node 13 (127.7027545, 34.7678666) : Demand 385
Node 14 (127.7049647, 34.7694184) : Demand 670
Node 15 (127.73225, 34.9403013) : Demand 56
Node 16 (126.4345523, 34.8042155) : Demand 607
Node 17 (126.7719252, 34.6443175) : Demand 255
Node 18 (127.725888, 34.741151) : Demand 634
Node 19 (127.6510494, 34.7737838) : Demand 571
Node 20 (127.1409901, 35.2755645) : Demand 385
Node 21 (127.720362, 34.7621065) : 

In [8]:
from itertools import combinations

for n1, n2 in tqdm([comb for comb in combinations(g.nodes, 2)]):
    if n1.index > n2.index:
        temp = n2
        n2 = n1
        n1 = temp

    link_cost = get_cost(n1.index, n2.index, COST_TYPE, JSON_PATH, toll_include=True)
    l = Link(n1, n2, link_cost)
    g.add_link(l)

100%|██████████| 1431/1431 [00:10<00:00, 142.36it/s]


In [9]:
print(g.get_link(53, 4))

Link (4 - 53) : Cost 4932


In [11]:
import pickle
with open('./../Data/graph.pkl', 'wb') as file:
    pickle.dump(g, file)

### node간 cost matrix

In [108]:
DEPOT_INDEX = 0
COST_TYPE = 'trafast'
JSON_PATH = './../Data/routes'

node_data = pd.read_csv('./../Data/data.csv')
nodes = node_data[node_data['index']!=DEPOT_INDEX]['index'].values

# cost_matrix는 실제 node 두 개에 대해서는 유의미한 값, 아닌 node 조합에는 -1을 저장
cost_matrix = np.full((max(nodes)+1, max(nodes)+1), -1)

for i in tqdm(nodes):
    for j in nodes:
        if i == j:
            cost_matrix[i][i] = 0
            continue
        elif i > j:
            continue
        # i < j일 때에만 계산 후 반대쪽에도 채우기
        else:
            cost_matrix[i][j] = cost_matrix[j][i] = get_cost(i, j, COST_TYPE, JSON_PATH, toll_include=True)

100%|██████████| 53/53 [00:09<00:00,  5.35it/s]


In [109]:
# 배열을 파일로 저장
np.save('./../Data/cost_matrix.npy', cost_matrix)